BIO1 = Annual Mean Temperature
BIO2 = Mean Diurnal Range (Mean of monthly (max temp - min temp))
BIO3 = Isothermality (BIO2/BIO7) (* 100)
BIO4 = Temperature Seasonality (standard deviation *100)
BIO5 = Max Temperature of Warmest Month
BIO6 = Min Temperature of Coldest Month
BIO7 = Temperature Annual Range (BIO5-BIO6)
BIO8 = Mean Temperature of Wettest Quarter
BIO9 = Mean Temperature of Driest Quarter
BIO10 = Mean Temperature of Warmest Quarter
BIO11 = Mean Temperature of Coldest Quarter
BIO12 = Annual Precipitation
BIO13 = Precipitation of Wettest Month
BIO14 = Precipitation of Driest Month
BIO15 = Precipitation Seasonality (Coefficient of Variation)
BIO16 = Precipitation of Wettest Quarter
BIO17 = Precipitation of Driest Quarter
BIO18 = Precipitation of Warmest Quarter
BIO19 = Precipitation of Coldest Quarter

In [1]:
import pandas as pd

In [2]:
# download temperature source
src = '../src/orchard_house-temp_hourly.csv'
df = pd.read_csv(src, parse_dates=['datetime'], index_col='datetime')

In [3]:
# subselect downloaded columns for basic temperature columns
df = df.iloc[:, [1, 4, 7, 10, 13, 16]]

In [4]:
# rename columns
df.columns = ['Baldy Summit (°F)',
 'Baldy Draw (°F)',
 'Sanfoin Bench (°F)',
 'South Baldy (°F)',
 'Orchard House (°F)',
 'Indian Ridge (°F)']

In [5]:
# average all stations columns together by row
station_ave = df.mean(axis=1).to_frame(name='temp')

In [6]:
# 2012 and 2019 are incomplete, remove them
station_ave = station_ave['2013':'2018']

In [7]:
station_ave.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 140509 entries, 2013-01-01 00:00:00 to 2018-12-31 23:30:00
Data columns (total 1 columns):
temp    140486 non-null float64
dtypes: float64(1)
memory usage: 2.1 MB


In [8]:
station_ave.head(2)

,temp
datetime,
2013-01-01 00:00:00,18.416667
2013-01-01 00:05:00,18.266667


### BIO1 = Annual Mean Temperature

In [9]:
BIO1 = station_ave.groupby(pd.Grouper(freq='Y')).mean().round(1)

In [10]:
BIO1

,temp
datetime,
2013-12-31,56.3
2014-12-31,44.8
2015-12-31,47.3
2016-12-31,45.9
2017-12-31,44.7
2018-12-31,44.4


In [11]:
BIO1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6 entries, 2013-12-31 to 2018-12-31
Freq: A-DEC
Data columns (total 1 columns):
temp    6 non-null float64
dtypes: float64(1)
memory usage: 96.0 bytes


### BIO2 = Mean Diurnal Range (Mean of monthly (max temp - min temp))

In [12]:
max_temp = station_ave.groupby(pd.Grouper(freq='M')).max()
min_temp = station_ave.groupby(pd.Grouper(freq='M')).min()
BIO2 = max_temp - min_temp

In [13]:
BIO2.head()

,temp
datetime,
2013-01-31,37.700000
2013-02-28,NaN
2013-03-31,NaN
2013-04-30,39.250000
2013-05-31,53.983333


In [14]:
BIO2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72 entries, 2013-01-31 to 2018-12-31
Freq: M
Data columns (total 1 columns):
temp    70 non-null float64
dtypes: float64(1)
memory usage: 1.1 KB


### BIO3 = Isothermality (BIO2/BIO7) (* 100)

In [15]:
# need BIO7

### BIO4 = Temperature Seasonality (standard deviation *100)

### BIO5 = Max Temperature of Warmest Month

In [18]:
# identify  warmest month by year
warmest_months = []
years = station_ave.index.year.unique().tolist()

for year in years:
    warmest = 0
    # average temp values by month
    month_ave = station_ave[str(year)].groupby(pd.Grouper(freq='M')).mean()
    
    # loop through monthly averages and select warmest month year
    for index, row in month_ave.iterrows():
        if row.temp > warmest:
            warmest = row.temp
            year = index.year
            month = index.month
    warmest_month = {'year': year, 'month': month, 'warmest_average': warmest.round(1)}
    warmest_months.append(warmest_month)

In [19]:
warmest_months

[{'year': 2013, 'month': 7, 'warmest_average': 71.7},
 {'year': 2014, 'month': 7, 'warmest_average': 70.8},
 {'year': 2015, 'month': 6, 'warmest_average': 67.4},
 {'year': 2016, 'month': 8, 'warmest_average': 65.2},
 {'year': 2017, 'month': 7, 'warmest_average': 72.7},
 {'year': 2018, 'month': 7, 'warmest_average': 68.3}]

In [20]:
# select max value from warmest month by year
BIO5 = []
for value in warmest_months:
    year = str(value['year'])
    month = str(value['month'])
    max_value = station_ave[year + '-' + month].max()
    BIO5.append({'max_temp': max_value.temp.round(1), 'year': year, 'month': month})

In [21]:
BIO5

[{'max_temp': 92.1, 'year': '2013', 'month': '7'},
 {'max_temp': 89.8, 'year': '2014', 'month': '7'},
 {'max_temp': 94.3, 'year': '2015', 'month': '6'},
 {'max_temp': 87.9, 'year': '2016', 'month': '8'},
 {'max_temp': 95.6, 'year': '2017', 'month': '7'},
 {'max_temp': 86.7, 'year': '2018', 'month': '7'}]

In [22]:
BIO5 = pd.DataFrame(BIO5)

### BIO6 = Min Temperature of Coldest Month

In [23]:
# identify coldest month by year
coldest_months = []
years = station_ave.index.year.unique().tolist()

for year in years:
    coldest = 9999
    # average temp values by month
    month_ave = station_ave[str(year)].groupby(pd.Grouper(freq='M')).mean()
    
    # loop through monthly averages and find coldest month by year
    for index, row in month_ave.iterrows():    
        if row.temp < coldest:
            coldest = row.temp
            year = index.year
            month = index.month
    coldest_month = {'year': year, 'month': month, 'coldest_average': coldest.round(1)}
    coldest_months.append(coldest_month)

In [24]:
coldest_months

[{'year': 2013, 'month': 1, 'coldest_average': 20.7},
 {'year': 2014, 'month': 2, 'coldest_average': 19.5},
 {'year': 2015, 'month': 12, 'coldest_average': 26.9},
 {'year': 2016, 'month': 12, 'coldest_average': 20.3},
 {'year': 2017, 'month': 1, 'coldest_average': 18.3},
 {'year': 2018, 'month': 2, 'coldest_average': 24.1}]

In [25]:
# select min value from coldest month by year
BIO6 = []
for value in coldest_months:
    year = str(value['year'])
    month = str(value['month'])
    min_temp = station_ave[year + '-' + month].min()
    BIO6.append({'min_temp': min_temp.temp.round(1), 'year': year, 'month': month})

In [26]:
BIO6

[{'min_temp': 3.8, 'year': '2013', 'month': '1'},
 {'min_temp': -16.3, 'year': '2014', 'month': '2'},
 {'min_temp': 11.1, 'year': '2015', 'month': '12'},
 {'min_temp': -7.5, 'year': '2016', 'month': '12'},
 {'min_temp': -9.4, 'year': '2017', 'month': '1'},
 {'min_temp': -3.4, 'year': '2018', 'month': '2'}]

In [27]:
BIO6 = pd.DataFrame(BIO6)

In [28]:
BIO6

,min_temp,month,year
0,3.8,1,2013
1,-16.3,2,2014
2,11.1,12,2015
3,-7.5,12,2016
4,-9.4,1,2017
5,-3.4,2,2018


### BIO7 = Temperature Annual Range (BIO5-BIO6)

In [29]:
BIO7 = (BIO5.max_temp - BIO6.min_temp).round(1).to_list()

In [30]:
BIO7 = []

for index, row in BIO5.iterrows():
    temp_range = BIO5.loc[index, 'max_temp'] - BIO6.loc[index, 'min_temp']
    year = row.year
    BIO7.append({'year': year, 'temp_range': temp_range.round(1)})

In [31]:
BIO7

[{'year': '2013', 'temp_range': 88.3},
 {'year': '2014', 'temp_range': 106.1},
 {'year': '2015', 'temp_range': 83.2},
 {'year': '2016', 'temp_range': 95.4},
 {'year': '2017', 'temp_range': 105.0},
 {'year': '2018', 'temp_range': 90.1}]

### BIO8 = Mean Temperature of Wettest Quarter